<a href="https://colab.research.google.com/github/JSK2022/RandomForest/blob/test/JS_simulation_code_230824.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-survival

In [ ]:
class RisksetCounter:
    def __init__(self, ids, time_start, time_stop, event):
        self.ids = ids
        self.time_start = time_start
        self.time_stop = time_stop
        self.event = event

        # \( t_1, t_2, \dots, t_n \): n distinct unique event times
        self.all_unique_times = np.unique(time_stop)
        self.n_unique_times = len(self.all_unique_times)

        # Initialize arrays to store the number at risk and number of events at each unique time
        self.n_at_risk = np.zeros(self.n_unique_times, dtype=np.int64)
        self.n_events = np.zeros(self.n_unique_times, dtype=np.int64)

        # Get unique times and events by ID
        self.unique_times_by_id, self.has_event_by_id = self.get_unique_times_by_id()

        # Populate the at-risk and events arrays
        self.set_data()

    def get_unique_times_by_id(self):
        unique_times_by_id = defaultdict(list)
        has_event_by_id = defaultdict(list)

        for id_, t_start, t_stop, e in zip(self.ids, self.time_start, self.time_stop, self.event):
            if not unique_times_by_id[id_] or t_stop != unique_times_by_id[id_][-1]:
                unique_times_by_id[id_].append(t_stop)
                has_event_by_id[id_].append(e)
            elif e == 1:
                has_event_by_id[id_][-1] = 1

        for id_ in unique_times_by_id.keys():
            unique_times_by_id[id_] = np.asarray(unique_times_by_id[id_])
            has_event_by_id[id_] = np.asarray(has_event_by_id[id_], dtype=np.int64)

        return unique_times_by_id, has_event_by_id

    def reset(self):
        self.n_at_risk.fill(0)
        self.n_events.fill(0)
        self.set_data()

    def set_data(self):
        for id_, times in self.unique_times_by_id.items():
            events = self.has_event_by_id[id_]
            for t, e in zip(times, events):
                idx = np.searchsorted(self.all_unique_times, t)
                self.n_at_risk[idx:] += 1
                self.n_events[idx] += e

    def update(self, ids, time_start, time_stop, event):
        new_unique_times_by_id, new_has_event_by_id = self.get_unique_times_by_id()

        for id_, times in new_unique_times_by_id.items():
            events = new_has_event_by_id[id_]
            for t, e in zip(times, events):
                idx = np.searchsorted(self.all_unique_times, t)
                self.n_at_risk[idx:] -= 1
                self.n_events[idx] -= e

    def at_id_time(self, id_, t_idx):
        at_risk = 0
        events = 0

        times_for_id = self.unique_times_by_id.get(id_, [])
        events_for_id = self.has_event_by_id.get(id_, [])

        time_at_t_idx = self.all_unique_times[t_idx]
        if time_at_t_idx in times_for_id:
            idx_in_id_times = np.searchsorted(times_for_id, time_at_t_idx)
            at_risk = 1
            events = events_for_id[idx_in_id_times]

        return at_risk, events

    def Y_i(self, id_, t_idx):
        # \( Y_i(t) \) for individual i at time t_idx
        time_at_t_idx = self.all_unique_times[t_idx]
        times_for_id = self.unique_times_by_id.get(id_, [])
        tau_i = times_for_id[-1] if times_for_id else 0
        return 1 if time_at_t_idx <= tau_i else 0

    def dN_bar_i(self, id_, t_idx):
        # \( d\bar{N}_i(t) \) for individual i at time t_idx
        Y_i_t = self.Y_i(id_, t_idx)
        _, event = self.at_id_time(id_, t_idx)
        return Y_i_t * event

    def N_bar_i(self, id_, t_idx):
        # \( \bar{N}_i(t) \) for individual i at time t_idx
        return np.sum([self.dN_bar_i(id_, idx) for idx in range(t_idx + 1)])

    def Y(self, t_idx):
        # Corrected calculation for \( Y_{\cdot}(t) \):
        # total number of subjects at risk ONLY at the interval ending at t_idx
        return self.n_at_risk[t_idx]

    def dN_bar(self, t_idx):
        # \( d\bar{N}_{\cdot}(t) \): total number of events observed over [t, t+dt)
        return self.n_events[t_idx]

    def N_bar(self, t_idx):
        # Cumulative number of events observed from start to t
        return np.sum(self.n_events[:t_idx + 1])

# To demonstrate, let's create an instance of this class
ids = [1, 2, 3, 1]
time_start = [0, 0, 0, 1]
time_stop = [1, 2, 3, 2]
event = [1, 0, 1, 1]

risk_counter = RisksetCounter(ids, time_start, time_stop, event)

# Return some example calculations
risk_counter.Y(1), risk_counter.dN_bar(1), risk_counter.N_bar(1)


1. 초기화('__ __init__ __' 메서드)
  * 입력데이터로 ID, 시작 시간, 종료 시간, 사건 발생 여부를 받는다.
  * 모든 고유한 종료 시간을 정렬하여 저장한다.
  * 각 고유한 시간에 대한 위험 집합과 사건 수를 저장하는 배열을 초기화한다.
  * 각 ID에 대한 고유한 시간과 사건 발생 여부를 계산하고 저장한다.
  * 위험 집합과 사건 수를 설정한다.

2. 각 ID에 대한 고유한 시간과 사건 정보 추출 ('get_unique_times_by_id' 메서드)
  * 각 ID에 대해 고유한 종료 시간과 해당 시간에 사건이 발생했는지를 저장한다.

3. 데이터 설정('set_data' 메서드)
  * 각 ID와 그에 해당하는 시간에 대해, 위험 집합과 사건 수를 업데이트 한다.

4. 데이터 업데이트 ('update' 메서드)
  * 새로운 데이터가 주어졌을 때 위험 집합과 사건 수를 업데이트 한다.

5. 특정ID와 시간에 대한 위험 집합과 사건 정보('at_id_time' 메서드)
  * 주어진 ID와 시간에 해당하는 위험 집합과 사건 정보를 반환한다.

6. 다양한 통계적 계산 메서드 ('Y_i','dN_bar_i','N_bar_i','Y','dN_bar','N_bar')
  * 주어진 시간과 ID에 대한 통계적 정보를 계산한다.

In [ ]:
import numpy as np
import pandas as pd
data = pd.read_csv("/content/bladder2.csv")

In [ ]:
ids_recurrent=np.array(data['id'])
time_start_recurrent=np.array(data['start'])
time_stop_recurrent=np.array(data['stop'])
event_recurrent=np.array(data['event'])

In [ ]:
data[['rx','number','size','enum']].values

In [ ]:
# Initialize the CombinedRisksetCounter with the recurrent event data
riskset_counter_recurrent = RisksetCounter(ids_recurrent, time_start_recurrent, time_stop_recurrent, event_recurrent)

In [ ]:
# Return some example calculations
riskset_counter_recurrent.Y(50), riskset_counter_recurrent.dN_bar(1), riskset_counter_recurrent.N_bar(1)

In [ ]:
# Example recurrent event data
ids_example = ['A', 'A', 'B', 'C', 'D']
time_start_example = [0, 2, 0, 0, 1]
time_stop_example = [1, 3, 2, 3, 3]
event_example = [1, 1, 0, 1, 0]

# Initialize the RisksetCounter with the example data
risk_counter_example = RisksetCounter(ids_example, time_start_example, time_stop_example, event_example)

# Calculate Y and dN_bar for each unique time point
Y_values_example = [risk_counter_example.Y(i) for i in range(risk_counter_example.n_unique_times)]
dN_bar_values_example = [risk_counter_example.dN_bar(i) for i in range(risk_counter_example.n_unique_times)]

Y_values_example, dN_bar_values_example

위의 재발 사건 데이터 예시를 바탕으로 계산한 결과는 다음과 같습니다:

1. $t_1$에서 위험 집합에 있는 대상의 수: 1명 (환자 A), 이 시점에서 발생한 이벤트의 수: 1(환자 A)
2. $t_2$에서 위험 집합에 있는 대상의 수는 2명(A,B) 그리고 이 시점에서 발생한 이벤트의 수 0
3. $t_3$에서 위험집합에 있는 대상의 수는 5명 (A, B, C, D 및 환자 A의 재발) 그리고 이 시점에서 발생한 이벤트의 수는 2 (A, C)


In [ ]:
# Example recurrent event data
ids_example = [1,1,1,2,2,3,3,3,3]
time_start_example = [0, 1, 3, 0, 2, 0, 3, 5, 8]
time_stop_example = [1, 3, 6, 2, 5, 3, 5, 8, 12]
event_example = [1, 1, 0, 1, 0, 1, 1, 1, 0]

# Initialize the RisksetCounter with the example data
risk_counter_example = RisksetCounter(ids_example, time_start_example, time_stop_example, event_example)

# Calculate Y and dN_bar for each unique time point
Y_values_example = [risk_counter_example.Y(i) for i in range(risk_counter_example.n_unique_times)]
dN_bar_values_example = [risk_counter_example.dN_bar(i) for i in range(risk_counter_example.n_unique_times)]

Y_values_example, dN_bar_values_example

In [ ]:
risk_counter_example.n_unique_times

In [ ]:
def argbinsearch(arr, key_val):
    arr_len = len(arr)
    min_idx = 0
    max_idx = arr_len

    while min_idx < max_idx:
        mid_idx = min_idx + ((max_idx - min_idx) // 2)

        if mid_idx < 0 or mid_idx >= arr_len:
            return -1

        mid_val = arr[mid_idx]
        if mid_val <= key_val:  # Change the condition to <=
            min_idx = mid_idx + 1
        else:
            max_idx = mid_idx

    return min_idx

이 함수는 argbinsearch라는 이름의 함수로, 배열에서 주어진 키 값보다 크거나 같은 첫 번째 원소의 인덱스를 이진 탐색으로 찾아 반환합니다.

자세한 코드 설명을 아래에 제공합니다:

1. 입력:

  * arr: 탐색 대상인 정렬된 배열
  key_val: 찾고자 하는 키 값

2. 초기 변수 설정:

  * arr_len: 배열의 길이를 저장합니다.
  * min_idx: 탐색 범위의 최솟값으로, 처음에는 배열의 시작 인덱스인 0으로 설정됩니다.
  * max_idx: 탐색 범위의 최댓값으로, 처음에는 배열의 길이로 설정됩니다.

3. 이진 탐색:

  * while 루프를 사용하여 min_idx가 max_idx보다 작은 동안 탐색을 반복합니다.
  * mid_idx: 현재 탐색 범위의 중간 인덱스를 계산합니다.
  * mid_val: 중간 인덱스에 해당하는 배열의 원소 값을 가져옵니다.

4. 키 값과 중간 값을 비교합니다:
  * 만약 중간 값이 키 값보다 작거나 같으면, min_idx를 mid_idx + 1로 업데이트합니다. 이렇게 하면 탐색 범위의 왼쪽 부분을 제외하게 됩니다.
  * 그렇지 않으면, max_idx를 mid_idx로 업데이트합니다. 이렇게 하면 탐색 범위의 오른쪽 부분을 제외하게 됩니다.

5. 결과 반환:

  * 루프가 종료되면, min_idx는 키 값보다 크거나 같은 첫 번째 원소의 인덱스를 가리키게 됩니다. 따라서 min_idx를 반환합니다.

이 함수는 정렬된 배열에서 주어진 키 값보다 크거나 같은 첫 번째 원소의 위치를 효율적으로 찾기 위해 사용됩니다. 이진 탐색은 배열의 중간 값을 반복적으로 확인하면서 탐색 범위를 절반씩 줄여나가므로, 큰 배열에서도 빠르게 원하는 값을 찾을 수 있습니다.

In [ ]:
class PseudoScoreCriterion:
    def __init__(self, n_outputs, n_samples, unique_times, x, ids, time_start, time_stop, event, random_state=None):
        self.n_outputs = n_outputs
        self.n_samples = n_samples
        self.n_unique_times = len(unique_times)

        self.x = x
        self.ids = ids
        self.time_start = time_start
        self.time_stop = time_stop
        self.event = event
        self.unique_times = unique_times
        self.random_state = check_random_state(random_state)

        self.riskset_total = RisksetCounter(ids, time_start, time_stop, event)

        self.Y_left = np.zeros(self.n_unique_times, dtype=np.int64)
        self.Y_right = np.zeros(self.n_unique_times, dtype=np.int64)
        self.dN_left = np.zeros(self.n_unique_times, dtype=np.int64)
        self.dN_right = np.zeros(self.n_unique_times, dtype=np.int64)

        self.samples_time_idx = np.zeros(n_samples, dtype=np.int64)
        for i in range(n_samples):
            self.samples_time_idx[i] = np.searchsorted(unique_times, time_stop[i])

    def init(self, y, sample_weight, n_samples, samples, start, end):
        start_times = y[:, 0]
        stop_times = y[:, 1]
        event = y[:, 2]
        self.samples = samples  # Storing the samples for this node

        for idx in samples[start:end]:
            self.riskset_total.update([self.ids[idx]], [start_times[idx]], [stop_times[idx]], [event[idx]])

    def update(self, new_pos, split_feature, split_threshold):
    # Initialize the statistics for each side of the split
      self.Y_left.fill(0)
      self.Y_right.fill(0)
      self.dN_left.fill(0)
      self.dN_right.fill(0)

      # Ensure that new_pos does not exceed the length of self.samples
      new_pos = min(new_pos, len(self.samples))

      for i in range(new_pos):
          idx = self.samples[i]
          event = self.event[idx]
          time_idx = self.samples_time_idx[idx]

          is_left = self.x[idx, split_feature] <= split_threshold

          if is_left:
              self.Y_left[time_idx] += 1
              self.dN_left[time_idx] += event
          else:
              self.Y_right[time_idx] += 1
              self.dN_right[time_idx] += event

    def proxy_impurity_improvement(self):
        w = (self.Y_left * self.Y_right) / (self.Y_left + self.Y_right + 1e-7)
        numer = np.sum(w * (self.dN_left / (self.Y_left + 1e-7) - self.dN_right / (self.Y_right + 1e-7)))

        var_estimate = 0.0
        for t in range(self.n_unique_times):
            for Y, dN in [(self.Y_left, self.dN_left), (self.Y_right, self.dN_right)]:
                if Y[t] == 0:
                    continue
                term = w[t] * Y[t] / (self.Y_left[t] + self.Y_right[t]) * (dN[t] - (self.dN_left[t] + self.dN_right[t]) / (self.Y_left[t] + self.Y_right[t]))
                var_estimate += term ** 2

        if var_estimate != 0.0:
            return numer / np.sqrt(var_estimate + 1e-7)
        else:
            return numer

    def node_value(self):
        # The Nelson-Aalen estimator for the entire node
        return np.cumsum(self.dN_left + self.dN_right) / (self.Y_left + self.Y_right + 1e-7)

    def reset(self):
        self.riskset_total.reset()

    def copy(self):
        """Create a deep copy of the criterion object."""
        new_criterion = PseudoScoreCriterion(self.n_outputs, self.n_samples, self.unique_times,
                                             self.x, self.ids, self.time_start, self.time_stop,
                                             self.event, self.random_state)
        new_criterion.Y_left = self.Y_left.copy()
        new_criterion.Y_right = self.Y_right.copy()
        new_criterion.dN_left = self.dN_left.copy()
        new_criterion.dN_right = self.dN_right.copy()
        new_criterion.samples_time_idx = self.samples_time_idx.copy()
        # Copy samples attribute if it exists
        if hasattr(self, 'samples'):
            new_criterion.samples = self.samples.copy()
        return new_criterion

# The modified PseudoScoreCriterion class is now ready for testing or further utilization.


## Standardized pseudo score criterion을 지정하는 함수

1. 초기화 (__init__ 메서드):

  - 입력 변수로 총 출력 수, 샘플 수, 고유한 시간 값, 입력 특성, ID, 시작 시간, 종료 시간, 사건, 난수 생성 상태를 받습니다.
  - 주어진 데이터를 이용하여 RisksetCounter 인스턴스를 생성합니다. 이 인스턴스는 각 시간에서의 위험 집합과 사건 수를 계산합니다.
  - 각 시간에서의 위험 집합과 사건 수를 저장하기 위한 배열을 초기화합니다.
  - 각 샘플에 대한 고유한 시간 인덱스를 계산하여 저장합니다.

2. 초기 설정 (init 메서드):

  - 주어진 샘플 범위에 대해 RisksetCounter 인스턴스를 업데이트합니다.

3. 업데이트 (update 메서드):

  - 주어진 분할 기준에 따라 각 샘플이 왼쪽 노드 또는 오른쪽 노드로 분할되는지를 판단하고, 해당 노드의 위험 집합과 사건 수를 업데이트합니다.

4. 임퓨리티 향상 추정 (proxy_impurity_improvement 메서드):

  - 주어진 분할에 대한 임퓨리티 향상을 추정합니다. 이 추정치는 분할의 품질을 평가하는 데 사용됩니다.

5. 노드 값 계산 (node_value 메서드):

  - 전체 노드에 대한 누적 위험 함수의 추정치를 계산합니다.

6. 재설정 (reset 메서드):

  - RisksetCounter 인스턴스를 초기 상태로 재설정합니다.

7. 복사 (copy 메서드):

  - PseudoScoreCriterion 인스턴스의 깊은 복사본을 생성합니다.

In [ ]:
# 필요한 라이브러리 및 함수 임포트
import numpy as np
from sklearn.utils import check_random_state

def check_random_state(seed):
    if seed is None or isinstance(seed, (int, np.integer)):
        return np.random.RandomState(seed)
    elif isinstance(seed, np.random.RandomState):
        return seed
    else:
        raise ValueError("seed must be None, int or np.random.RandomState")

# (앞서 제공된 RisksetCounter 클래스 코드)

# (앞서 제공된 PseudoScoreCriterion 클래스 코드)

# 데이터 생성
n_samples = 100
n_features = 2

np.random.seed(42)

x = np.random.rand(n_samples, n_features)
ids = np.arange(n_samples)
time_start = np.zeros(n_samples)
time_stop = np.random.randint(1, 5, size=n_samples)
event = np.random.randint(0, 2, size=n_samples)

# PseudoScoreCriterion 객체 초기화
unique_times = np.unique(time_stop)
criterion = PseudoScoreCriterion(n_outputs=1, n_samples=n_samples, unique_times=unique_times,
                                 x=x, ids=ids, time_start=time_start, time_stop=time_stop,
                                 event=event, random_state=42)

# y 배열 생성
y = np.column_stack([time_start, time_stop, event])

# init 메서드 호출
samples = np.arange(n_samples)
criterion.init(y, sample_weight=None, n_samples=n_samples, samples=samples, start=0, end=n_samples)

# 임의의 분할 기준에 따라 update 메서드 호출
split_feature = 0
split_threshold = 0.5
new_pos = n_samples // 2
criterion.update(new_pos, split_feature, split_threshold)

# 분할의 임퓨리티 향상도 계산
impurity_improvement = criterion.proxy_impurity_improvement()
print(f"Impurity Improvement: {impurity_improvement}")

# 노드의 값 계산
node_val = criterion.node_value()
print(f"Node Value: {node_val}")


$\textbf{PseudoScoreCriterion}$ 클래스를 사용하여 주어진 분할 기준에 따른 손실 함수의 변화를 계산한 결과는 0.0입니다.

이는 선택한 분할 기준이 손실 함수를 개선하지 않았음을 의미합니다. 다른 분할 기준을 시도하면 다른 결과를 얻을 수 있습니다.

이 코드 예시를 통해 PseudoScoreCriterion 클래스가 정상적으로 작동함을 확인할 수 있습니다.

In [ ]:
data

In [ ]:
ids=np.array(data['id'])
time_start=np.array(data['start'])
time_stop=np.array(data['stop'])
event=np.array(data['event'])
X=data[['rx','number','size','enum']].values

In [ ]:
# Initialize the RisksetCounter with the example data
risk_counter = RisksetCounter(ids, time_start, time_stop, event)

# Calculate Y and dN_bar for each unique time point
Y_values = [risk_counter.Y(i) for i in range(risk_counter.n_unique_times)]
dN_bar_values = [risk_counter.dN_bar(i) for i in range(risk_counter.n_unique_times)]

Y_values, dN_bar_values

In [ ]:
n_samples_example = len(ids)
y_example = np.column_stack([time_start, time_stop, event])
samples_example = np.arange(n_samples_example)
sample_weight_example = None

# 2. Initialize PseudoScoreCriterion object
criterion_example = PseudoScoreCriterion(n_outputs=1,
                                         n_samples=n_samples_example,
                                         unique_times=risk_counter.all_unique_times,
                                         x=X,
                                         ids=ids,
                                         time_start=time_start,
                                         time_stop=time_stop,
                                         event=event,
                                         random_state=42)

criterion_example.init(y_example, sample_weight_example, n_samples_example, samples_example, 0, n_samples_example)

# 3. Set a random split criterion
split_feature = 0
split_threshold = 5

# 4. Update the criterion based on the split
criterion_example.update(n_samples_example // 2, 1, 3)

# 5. Calculate the proxy impurity improvement
proxy_impurity_improvement_example = criterion_example.proxy_impurity_improvement()

proxy_impurity_improvement_example

criterion_example.node_value()

In [ ]:
!pip install scikit-survival

In [ ]:
import pandas as pd

class PseudoScoreTreeBuilder:
    TREE_UNDEFINED = -1  # Placeholder

    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1, random_state=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.random_state = random_state

    def _split(self, X, criterion, start, end):
        """Find the best split for a node."""
        best_split = {
            'feature_index': None,
            'threshold': None,
            'improvement': -np.inf,
            'dN_bar_left': None,
            'Y_left': None,
            'dN_bar_right': None,
            'Y_right': None
        }

        # For each feature
        for feature_index in range(X.shape[1]):
            # Sort samples based on the feature values
            sorted_indices = np.argsort(X[start:end, feature_index])
            X_sorted = X[start:end][sorted_indices]

            # For each possible split threshold
            for i in range(1, len(X_sorted)):
                # Avoid duplicate feature values
                if X_sorted[i, feature_index] == X_sorted[i - 1, feature_index]:
                    continue

                # Divide samples into two groups
                left_indices = sorted_indices[:i]
                right_indices = sorted_indices[i:]

                # Update the criterion with the new split
                criterion.update(new_pos=i, split_feature=feature_index, split_threshold=X_sorted[i, feature_index])

                # Compute the proxy impurity improvement
                improvement = criterion.proxy_impurity_improvement()

                # Check if this split is the best so far
                if improvement > best_split['improvement']:
                    best_split = {
                        'feature_index': feature_index,
                        'threshold': X_sorted[i, feature_index],
                        'improvement': improvement,
                        'dN_bar_left': criterion.dN_left,
                        'Y_left': criterion.Y_left,
                        'dN_bar_right': criterion.dN_right,
                        'Y_right': criterion.Y_right
                    }

        return best_split

    def _build(self, X, y, criterion, depth=0, start=0, end=None):
        n_samples = X.shape[0]
        if end is None:
            end = n_samples

        # Conditions for terminal node
        if depth == self.max_depth or (end - start) <= self.min_samples_leaf or (end - start) < self.min_samples_split:
            return {
                'feature': None,
                'threshold': None,
                'left_child': None,
                'right_child': None,
                'dN_bar': criterion.dN_left + criterion.dN_right,
                'Y': criterion.Y_left + criterion.Y_right
            }

        # Initialize the criterion with the samples in the current node
        criterion.init(y, None, n_samples, np.arange(start, end), start, end)

        # Find the best split
        best_split = self._split(X, criterion, start, end)
        if best_split['improvement'] == -np.inf:
            return {
                'feature': None,
                'threshold': None,
                'left_child': None,
                'right_child': None,
                'dN_bar': criterion.dN_left + criterion.dN_right,
                'Y': criterion.Y_left + criterion.Y_right
            }

        # Split the data based on the best split
        left_indices = np.where(X[start:end, best_split['feature_index']] <= best_split['threshold'])[0]
        right_indices = np.where(X[start:end, best_split['feature_index']] > best_split['threshold'])[0]

        # Recursively build the left and right subtrees
        left_child = self._build(X[left_indices], y[left_indices], criterion, depth=depth+1)
        right_child = self._build(X[right_indices], y[right_indices], criterion, depth=depth+1)

        return {
            'feature': best_split['feature_index'],
            'threshold': best_split['threshold'],
            'left_child': left_child,
            'right_child': right_child,
            'dN_bar': best_split['dN_bar_left'] + best_split['dN_bar_right'],
            'Y': best_split['Y_left'] + best_split['Y_right']
        }

    def build(self, X, ids, time_start, time_stop, event):
        n_samples, n_features = X.shape
        y = np.c_[time_start, time_stop, event]

        unique_times = np.unique(time_stop)
        criterion = PseudoScoreCriterion(n_outputs=n_features, n_samples=n_samples,
                                         unique_times=unique_times, x=X, ids=ids,
                                         time_start=time_start, time_stop=time_stop, event=event,
                                         random_state=self.random_state)

        # Build the tree
        tree = self._build(X, y, criterion)

        # Convert tree dictionary to dataframe for consistency
        tree_df = pd.DataFrame([tree])
        return tree_df


## PseudoScoreCriterion을 기반으로 tree를 build 하는 클래스
1. 클래스 초기와 ('__ init __')
  * 트리의 최대 깊이(max_depth), 분할을 시작하기 위한 최소 샘플 수(min_samples_split), 리프 노드가 되기 위한 최소 샘플 수(min_samples_leaf), 랜덤 상태(random_state) 등 트리의 주요 하이퍼파라미터를 정의
2. _split 함수:
  * 분할의 특정 기준에 따라 주어진 데이터의 하위 집합에 대해 최적의 분할을 찾는 함수
  * 각 특성에 대해 가능한 모든 분할 포인트를 살펴보고, 최적의 분할을 찾기 위해 각 분할의 quality를 평가
  * 최적의 분할은 feature의 index, value of threshold, 그리고 분할로 인한 품질 향상 등의 정보를 포함
3. _build 함수
  * 재귀적으로 트리를 구축하는 함수
  * 주어진 데이터에 대해 최적의 분할을 찾고, 이를 기반으로 왼쪽과 오른쪽 서브트리를 구축
  * 트리의 최대 깊이에 도달하거나, 리프 노드가 되기 위한 조건을 만족하면 종료
  * 각 노드: feature의 index, value of threshold, left/right daughter node, 노드의 데이터 통계를 포함하는 딕셔너리로 표현
4. build 함수
  * 사용자에게 제공되는 주요 함수로, 입력 데이터와 관련된 다양한 정보를 기반으로 트리를 구축
  * PseudoScoreCriterion은 트리 분할의 품질을 평가하는 데 사용되는 특정 기준을 나타냅니다. 이 기준은 시간적으로 연속된 데이터와 관련된 특정 통계를 계산하는 데 사용됩니다.
  * _build 함수를 사용하여 트리를 구축한 후, 결과 트리를 데이터프레임 형식으로 변환하여 반환합니다.

In [ ]:
# PseudoScoreTreeBuilder를 사용하여 트리를 구축합니다.
builder = PseudoScoreTreeBuilder(max_depth=3, min_samples_leaf=5, random_state=42)
tree_df = builder.build(X, ids, time_start, time_stop, event)

print(tree_df)

In [ ]:
class RecurrentTree:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1, random_state=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.random_state = random_state
        self.tree_ = None

    def fit(self, X, ids, time_start, time_stop, event, sample_weight=None):
        # Ensure input is in the expected format
        X = np.array(X)
        ids = np.array(ids)
        time_start = np.array(time_start)
        time_stop = np.array(time_stop)
        event = np.array(event)

        # Use the PseudoScoreTreeBuilder to build the tree
        builder = PseudoScoreTreeBuilder(
            max_depth=self.max_depth,
            min_samples_split=self.min_samples_split,
            min_samples_leaf=self.min_samples_leaf,
            random_state=self.random_state
        )
        self.tree_ = builder.build(X, ids=ids, time_start=time_start, time_stop=time_stop, event=event).iloc[0]

        return self

    def get_tree(self):
        """Return the tree as a dictionary."""
        return self.tree_

    def _traverse_tree(self, x, node):
        """Traverse the tree to find the terminal node for a given sample."""

        # Check if it's a terminal node
        if node["threshold"] is None:
            return node

        if x[node["feature"]] <= node["threshold"]:
            return self._traverse_tree(x, node["left_child"])  # Navigate to the left child
        else:
            return self._traverse_tree(x, node["right_child"])  # Navigate to the right child

    def predict_rate_function(self, X):
        """
        Predict the nonparametric estimates of dμ(t) = ρ(t)dt for given samples.
        """
        # Ensure input is in the expected format
        X = np.array(X)
        n_samples = X.shape[0]

        rate_functions = []
        for i in range(n_samples):
            # Traverse the tree to find the terminal node for the current sample
            terminal_node = self._traverse_tree(X[i], self.tree_)

            # Compute the nonparametric estimate for the rate function
            dN = terminal_node['dN_bar']
            Y = terminal_node['Y']
            rho_t = dN / (Y + 1e-7)
            rate_functions.append(rho_t)

        return rate_functions

    def predict_mean_function(self, X):
        """
        Predict the Nelson-Aalen estimator of the mean function for given samples.
        """
        # Ensure input is in the expected format
        X = np.array(X)
        n_samples = X.shape[0]

        mean_functions = []
        for i in range(n_samples):
            # Traverse the tree to find the terminal node for the current sample
            terminal_node = self._traverse_tree(X[i], self.tree_)

            # Compute the Nelson-Aalen estimator for the mean function
            dN = terminal_node['dN_bar']
            Y = terminal_node['Y']
            mu_t = np.cumsum(dN / (Y + 1e-7))
            mean_functions.append(mu_t)

        return mean_functions



## RecurrentTree

1. 초기화 (__ init __ 메서드):

초기화 시 최대 깊이(max_depth), 최소 리프 노드 크기(min_leaf), 그리고 난수 생성 상태(random_state)를 받습니다.
tree_는 학습된 트리를 저장하는 변수입니다.

2. 학습 (fit 메서드):

주어진 데이터(X, ids, time_start, time_stop, event)를 사용하여 트리를 학습합니다.
입력 데이터는 올바른 형식(numpy 배열)으로 변환됩니다.
PseudoScoreTreeBuilder를 사용하여 트리를 구축합니다. 이 클래스는 위에서 제공되지 않았기 때문에 실제 코드에서는 이 부분이 작동하지 않을 것입니다.
트리 가져오기 (get_tree 메서드):

학습된 트리를 딕셔너리 형태로 반환합니다.

3. 트리 순회 (_traverse_tree 메서드):

주어진 샘플(x)에 대해 트리를 순회하면서 해당 샘플이 속하는 종단 노드(리프 노드)를 찾습니다.

4. 위험률 함수 예측 (predict_rate_function 메서드):

주어진 샘플들에 대해 비모수적 위험률 함수의 추정치인
dμ(t)=ρ(t)dt를 예측합니다.

5. 평균 함수 예측 (predict_mean_function 메서드):

주어진 샘플들에 대해 Nelson-Aalen 추정치를 사용하여 평균 함수를 예측합니다.

In [ ]:
# 2. Train the RecurrentTree model
model = RecurrentTree(max_depth=5,random_state=42)
model.fit(X, ids, time_start, time_stop, event)


In [ ]:
model.predict_rate_function(X)

In [ ]:
model.predict_mean_function(X)

In [ ]:
model.get_tree()

In [ ]:
%pip install graphviz

In [ ]:
import numpy as np
from numbers import Integral, Real
from sklearn.utils import check_random_state

def _get_n_samples_bootstrap(n_ids, max_samples):
    """
    Modified for recurrent events. Get the number of IDs in a bootstrap sample.
    """
    if max_samples is None:
        return n_ids

    if isinstance(max_samples, Integral):
        if max_samples > n_ids:
            msg = "`max_samples` must be <= n_ids={} but got value {}"
            raise ValueError(msg.format(n_ids, max_samples))
        return max_samples

    if isinstance(max_samples, Real):
        return max(round(n_ids * max_samples), 1)

def _generate_sample_indices(random_state, ids, n_ids_bootstrap):
    """
    Sample unique IDs and then expand to all associated events.
    """
    random_instance = check_random_state(random_state)
    sampled_ids = np.random.choice(ids, n_ids_bootstrap, replace=True)
    return sampled_ids

def _generate_unsampled_indices(random_state, ids, n_ids_bootstrap):
    """
    Determine unsampled IDs and then expand to all associated events.
    """
    sampled_ids = _generate_sample_indices(random_state, ids, n_ids_bootstrap)
    unsampled_ids = np.setdiff1d(ids, sampled_ids)

    # Expand these unsampled IDs to include all their associated events.
    # Again, this will depend on your data structure.
    # As an example:
    # unsampled_indices = np.concatenate([events_by_id[id] for id in unsampled_ids])

    return unsampled_ids  # or return unsampled_indices based on your data structure



from warnings import catch_warnings, simplefilter
from sklearn.utils.class_weight import compute_sample_weight

def _parallel_build_trees(
    tree,
    bootstrap,
    X,
    y,
    ids,  # New parameter: a list/array of IDs corresponding to each event in X and y
    sample_weight,
    tree_idx,
    n_trees,
    verbose=0,
    class_weight=None,
    n_ids_bootstrap=None,  # Instead of n_samples_bootstrap
):
    """
    Private function used to fit a single tree in parallel for recurrent events."""
    if verbose > 1:
        print("building tree %d of %d" % (tree_idx + 1, n_trees))

    if bootstrap:
        unique_ids = np.unique(ids)
        n_ids = len(unique_ids)

        # Generate bootstrap samples using IDs
        sampled_ids = _generate_sample_indices(
            tree.random_state, unique_ids, n_ids_bootstrap
        )

        # Expand sampled IDs to all their associated events
        indices = np.where(np.isin(ids, sampled_ids))[0]

        if sample_weight is None:
            curr_sample_weight = np.ones((X.shape[0],), dtype=np.float64)
        else:
            curr_sample_weight = sample_weight.copy()

        # Adjust the sample weight based on how many times each ID was sampled
        sample_counts_for_ids = np.bincount(np.searchsorted(unique_ids, sampled_ids), minlength=n_ids)
        curr_sample_weight *= sample_counts_for_ids[np.searchsorted(unique_ids, ids)]

        if class_weight == "subsample":
            with catch_warnings():
                simplefilter("ignore", DeprecationWarning)
                curr_sample_weight *= compute_sample_weight("auto", y, indices=indices)
        elif class_weight == "balanced_subsample":
            curr_sample_weight *= compute_sample_weight("balanced", y, indices=indices)

        tree.fit(X[indices], y[indices], sample_weight=curr_sample_weight[indices], check_input=False)
    else:
        tree.fit(X, y, sample_weight=sample_weight, check_input=False)

    return tree


1. _get_n_samples_boostrap(n_is, max_samples)
 * Recurrent events를 위해 수정된 함수
 * 부트스트랩 샘플에 포함될 ID의 개수를 반환
2. _generate_sample_indices(random_state, ids, n_ids_bootstrap)
 * 고유한 ID들을 샘플링하고, 그 ID들과 관련된 모든 이벤트를 확장
 * 부트스트랩의 핵심 기능
3. _generate_unsampled_indices(random_state, ids, n_ids_bootstrap)
 * 샘플링되지 않은 ID를 결정하고, 이 ID와 관련된 모든 이벤트를 확장
4. _parallel_build_trees(...)
 * 병렬로 단일 트리를 구축하는 데 사용되는 주요 함수
 * 부트스트랩 방법을 사용하여 train data에서 샘플을 추출하고, 이 샘플을 사용하여 트리를 구축
 * 앙상블 모델에서 여러 트리를 동시에 훈련시키기 위함


In [ ]:
!pip install scikit-survival

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.utils import check_array, check_consistent_length
from sklearn.utils.metaestimators import available_if
from sklearn.utils.validation import check_is_fitted

from sksurv.exceptions import NoComparablePairException
from sksurv.nonparametric import CensoringDistributionEstimator, SurvivalFunctionEstimator
from sksurv.util import check_y_survival


def _check_estimate_1d(estimate, test_time):
    estimate = check_array(estimate, ensure_2d=False, input_name="estimate")
    if estimate.ndim != 1:
        raise ValueError(f"Expected 1D array, got {estimate.ndim}D array instead:\narray={estimate}.\n")
    check_consistent_length(test_time, estimate)
    return estimate

def _check_inputs(event_indicator, event_time, estimate):
    check_consistent_length(event_indicator, event_time, estimate)
    event_indicator = check_array(event_indicator, ensure_2d=False, input_name="event_indicator")
    event_time = check_array(event_time, ensure_2d=False, input_name="event_time")
    estimate = _check_estimate_1d(estimate, event_time)

    if not np.issubdtype(event_indicator.dtype, np.bool_):
        raise ValueError(
            f"only boolean arrays are supported as class labels for survival analysis, got {event_indicator.dtype}"
        )

    if len(event_time) < 2:
        raise ValueError("Need a minimum of two samples")

    if not event_indicator.any():
        raise ValueError("All samples are censored")

    return event_indicator, event_time, estimate


def _check_times(test_time, times):
    times = check_array(np.atleast_1d(times), ensure_2d=False, input_name="times")
    times = np.unique(times)

    if times.max() >= test_time.max() or times.min() < test_time.min():
        raise ValueError(
            f"all times must be within follow-up time of test data: [{test_time.min()}; {test_time.max()}["
        )

    return times

def _check_estimate_2d(estimate, test_time, time_points, estimator):
    estimate = check_array(estimate, ensure_2d=False, allow_nd=False, input_name="estimate", estimator=estimator)
    time_points = _check_times(test_time, time_points)
    check_consistent_length(test_time, estimate)

    if estimate.ndim == 2 and estimate.shape[1] != time_points.shape[0]:
        raise ValueError(f"expected estimate with {time_points.shape[0]} columns, but got {estimate.shape[1]}")

    return estimate, time_points


def _iter_comparable(event_indicator, event_time, order):
    n_samples = len(event_time)
    tied_time = 0
    i = 0
    while i < n_samples - 1:
        time_i = event_time[order[i]]
        end = i + 1
        while end < n_samples and event_time[order[end]] == time_i:
            end += 1

        # check for tied event times
        event_at_same_time = event_indicator[order[i:end]]
        censored_at_same_time = ~event_at_same_time
        for j in range(i, end):
            if event_indicator[order[j]]:
                mask = np.zeros(n_samples, dtype=bool)
                mask[end:] = True
                # an event is comparable to censored samples at same time point
                mask[i:end] = censored_at_same_time
                tied_time += censored_at_same_time.sum()
                yield (j, mask, tied_time)
        i = end

def _estimate_recurrent_concordance_index(event_times, event_counts, estimates):
    """
    Estimate the Concordance Index for recurrent events.

    Parameters:
    - event_times: numpy array of observation times (C_i and C_i') for each individual
    - event_counts: numpy array of event counts (N_i and N_i') for each individual
    - estimates: Out-of-Bag estimates for each individual (mu_OOB)

    Returns:
    - cindex: estimated concordance index
    """
    m = len(event_times)
    num = 0.0
    den = 0.0

    for i in range(m):
        for j in range(m):
            if i != j:
                combined_time = min(event_times[i], event_times[j])

                if event_counts[i][combined_time] > event_counts[j][combined_time]:
                    den += 1
                    if estimates[i][combined_time] > estimates[j][combined_time]:
                        num += 1

    cindex = num / den if den != 0 else 0
    return cindex

# Calculate the Prediction Error rate
def prediction_error_rate(cindex):
    return 1 - cindex


## C-Index

1. _check_estimate_1d, _check_estimate_2d, _check_inputs, _check_times:

  * 이 함수들은 입력 데이터의 유효성을 검사하는 유틸리티 함수
  * 주어진 입력 데이터의 일관성, 차원, 형식 등을 검사하여 데이터가 예상된 형식과 일치하는지 확인

2. _iter_comparable:
  * 주어진 이벤트 시간 및 지표에 대해 비교 가능한 샘플 조합을 반복하는 제너레이터 함수.
  * 이 함수는 생존 분석에서 두 샘플이 비교 가능한지를 결정하는 데 사용.

3. _estimate_recurrent_concordance_index:
  * 재발생 이벤트의 경우 Concordance Index (C-index)를 추정합니다.
  * 이 함수는 각 개체의 이벤트 시간, 이벤트 횟수, 그리고 각 개체에 대한 Out-of-Bag 추정치를 입력으로 받아 C-index를 계산.

4. prediction_error_rate:
  * 주어진 C-index를 기반으로 예측 오류율을 계산합니다.
  * 예측 오류율은 1 - C-index로 계산되며, 모델의 성능을 나타내는 또 다른 지표로 사용됨.

In [ ]:
from sklearn.ensemble import BaseEnsemble, BaggingRegressor
from sklearn.utils import check_random_state, resample
from sklearn.utils.validation import check_is_fitted

class RecurrentRandomForest:
    def __init__(self, n_estimators=100, max_depth=None, min_samples_split=2,
                 min_samples_leaf=1, bootstrap=True, oob_score=False, n_jobs=None,
                 random_state=None, verbose=0, warm_start=False, max_samples=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.bootstrap = bootstrap
        self.oob_score = oob_score
        self.n_jobs = n_jobs
        self.random_state = random_state
        self.verbose = verbose
        self.warm_start = warm_start
        self.max_samples = max_samples

        self.estimators_ = []
        for _ in range(self.n_estimators):
            tree = RecurrentTree(max_depth=self.max_depth,
                                 min_samples_split=self.min_samples_split,
                                 min_samples_leaf=self.min_samples_leaf,
                                 random_state=self.random_state)
            self.estimators_.append(tree)

    @property
    def feature_importances_(self):
        """Not implemented"""
        raise NotImplementedError()

    # Modify the fit method of RecurrentRandomForest to remove the DTYPE reference
    def fit(self, X, ids, time_start, time_stop, event, sample_weight=None):
        """Build a forest of survival trees from the training set (X, y)."""
        self._validate_params()

        X = self._validate_data(X, accept_sparse="csc", ensure_min_samples=2)

        # Validate the survival data
        event, time_start, time_stop = check_array_survival(X, (event, time_start, time_stop))

        self.n_features_in_ = X.shape[1]

        y = np.c_[time_start, time_stop, event]

        # Check parameters
        self._validate_estimator()

        if not self.bootstrap and self.oob_score:
            raise ValueError("Out of bag estimation only available if bootstrap=True")

        random_state = check_random_state(self.random_state)

        if not self.warm_start or not hasattr(self, "estimators_"):
            self.estimators_ = []

        n_more_estimators = self.n_estimators - len(self.estimators_)

        if n_more_estimators < 0:
            raise ValueError(
                f"n_estimators={self.n_estimators} must be larger or equal to "
                f"len(estimators_)={len(self.estimators_)} when warm_start==True"
            )

        trees = [self._make_estimator(append=False, random_state=random_state) for i in range(n_more_estimators)]

        # Parallel loop
        # Note: The actual Parallel and delayed functions are not implemented in this mock test.
        # So, the loop will just iterate over the trees normally.
        trees = [tree.fit(X, ids, time_start, time_stop, event, sample_weight, self.max_samples) for tree in trees]

        # Collect newly grown trees
        self.estimators_.extend(trees)

        if self.oob_score == True:
            # Note: OOB score computation for recurrent events might be more involved and is not covered here
            pass

        return self

    def _set_oob_score_and_attributes(self, X, y):
      """Calculate out of bag predictions and score."""
      n_samples = X.shape[0]
      event, time_start, time_stop = y  # Assuming y contains these three arrays.

      predictions = np.zeros(n_samples)
      n_predictions = np.zeros(n_samples)

      n_samples_bootstrap = _get_n_samples_bootstrap(n_samples, self.max_samples)

      for estimator in self.estimators_:
          unsampled_indices = _generate_unsampled_indices(estimator.random_state, n_samples, n_samples_bootstrap)
          p_estimator = estimator.predict(X[unsampled_indices, :], check_input=False)

          predictions[unsampled_indices] += p_estimator
          n_predictions[unsampled_indices] += 1

      if (n_predictions == 0).any():
          warnings.warn(
              "Some inputs do not have OOB scores. "
              "This probably means too few trees were used "
              "to compute any reliable oob estimates.",
              stacklevel=3,
          )
          n_predictions[n_predictions == 0] = 1

      predictions /= n_predictions
      self.oob_prediction_ = predictions

      # Use the new recurrent_concordance_index_censored function here.
      c_index = _estimate_recurrent_concordance_index(time_stop, event, predictions)
      self.oob_score_ = c_index


    def predict_rate_function(self, X):
        """
        Predict the nonparametric estimates of dμ(t) = ρ(t)dt for given samples using the forest.
        """
        check_is_fitted(self, "estimators_")
        X = self._validate_X_predict(X)

        # Parallel loop for rate function predictions
        rate_functions_results = Parallel(n_jobs=self.n_jobs, verbose=self.verbose, require="sharedmem")(
            delayed(tree.predict_rate_function)(X) for tree in self.estimators_
        )

        # Averaging rate functions results from all trees
        averaged_rate_functions = np.zeros((X.shape[0], len(rate_functions_results[0][0])))
        for rates in rate_functions_results:
            averaged_rate_functions += rates
        averaged_rate_functions /= len(self.estimators_)

        return averaged_rate_functions

    def predict_mean_function(self, X):
        """
        Predict the Nelson-Aalen estimator of the mean function for given samples using the forest.
        """
        check_is_fitted(self, "estimators_")
        X = self._validate_X_predict(X)

        # Parallel loop for mean function predictions
        mean_functions_results = Parallel(n_jobs=self.n_jobs, verbose=self.verbose, require="sharedmem")(
            delayed(tree.predict_mean_function)(X) for tree in self.estimators_
        )

        # Averaging mean functions results from all trees
        averaged_mean_functions = np.zeros((X.shape[0], len(mean_functions_results[0][0])))
        for means in mean_functions_results:
            averaged_mean_functions += means
        averaged_mean_functions /= len(self.estimators_)

        return averaged_mean_functions



## RandomForest for Recurrent Events

1. 클래스 초기화 (__init__):

  * 랜덤 포레스트의 주요 파라미터를 초기화합니다. 이러한 파라미터에는 트리의 개수(n_estimators), 최대 깊이(max_depth), 분할을 위한 최소 샘플 수(min_samples_split), 리프 노드의 최소 샘플 수(min_samples_leaf) 등이 포함됨
  * 또한, 주어진 파라미터를 기반으로 RecurrentTree 객체를 생성하여 estimators_ 리스트에 추가

2. fit 메서드:

  * 주어진 입력 데이터 X와 생존 데이터 (이벤트 지표, 시작 시간, 중지 시간)를 사용하여 랜덤 포레스트를 학습시킴
  * 각 트리는 병렬로 학습되며, 각 트리는 전체 데이터의 부트스트랩 샘플을 사용하여 학습됨
  * Out-of-bag (OOB) 점수를 계산할 경우 _set_oob_score_and_attributes 메서드를 호출하여 OOB 예측과 C-index를 계산

3. _set_oob_score_and_attributes 메서드:
  * Out-of-bag (OOB) 예측을 계산하고, 이를 기반으로 C-index를 계산
  * 이 메서드는 OOB 예측을 사용하여 모델의 성능을 추정하는 데 사용.

4. predict_rate_function 메서드:

  * 주어진 입력 데이터 X에 대한 비모수적 추정값 dμ(t)=ρ(t)dt를 예측.
  * 각 트리로부터의 비율 함수 예측을 병렬로 수집하고, 이러한 예측을 평균하여 최종 결과를 반환.

5. predict_mean_function 메서드:

  * 주어진 입력 데이터 X에 대한 Nelson-Aalen estiamator의 평균 함수를 예측.
  * 각 트리로부터의 평균 함수 예측을 병렬로 수집하고, 이러한 예측을 평균하여 최종 결과를 반환.


In [ ]:
RecurrentRandomForest(max_depth=10, random_state=42)

In [ ]:
# Generate synthetic recurrent survival data for testing
np.random.seed(1190)
n_samples = 100
n_features = 5

# Generate random data
X = np.random.randn(n_samples, n_features)
ids = np.arange(n_samples)
time_start = np.random.rand(n_samples) * 5
time_stop = time_start + np.random.rand(n_samples) * 5
event = np.random.randint(0, 2, n_samples)

X, ids, time_start, time_stop, event

In [ ]:
import numpy as np
from sklearn.utils.validation import check_array, check_X_y
from sklearn.utils.multiclass import check_classification_targets

# Mock version of check_array_survival
def check_array_survival(X, y_tuple):
    event, time_start, time_stop = y_tuple
    X = check_array(X)

    # Checking event, time_start, and time_stop
    check_classification_targets(event)
    time_start = check_array(time_start, ensure_2d=False)
    time_stop = check_array(time_stop, ensure_2d=False)

    if len(event) != len(time_start) or len(event) != len(time_stop):
        raise ValueError("event, time_start, and time_stop should have the same length.")

    return event, time_start, time_stop

# Mock version of _validate_params
def _validate_params(self):
    pass

# Mock version of _validate_data
def _validate_data(self, X, dtype=None, accept_sparse=None, ensure_min_samples=None):
    return check_array(X, dtype=dtype, accept_sparse=accept_sparse, ensure_min_samples=ensure_min_samples)

# Mock version of _validate_estimator
def _validate_estimator(self):
    pass

# Mock version of _make_estimator
def _make_estimator(self, append=True, random_state=None):
    # Creating a new instance of the RecurrentTree
    estimator = RecurrentTree(
        max_depth=self.max_depth,
        min_samples_split=self.min_samples_split,
        min_samples_leaf=self.min_samples_leaf,
        random_state=random_state
    )
    if append:
        self.estimators_.append(estimator)
    return estimator

# Mock version of _validate_X_predict
def _validate_X_predict(self, X):
    return check_array(X)

# Mock version of _get_n_samples_bootstrap
def _get_n_samples_bootstrap(n_samples, max_samples):
    return max_samples if max_samples is not None else n_samples

# Mock version of _generate_unsampled_indices
def _generate_unsampled_indices(random_state, n_samples, n_samples_bootstrap):
    sampled_mask = np.zeros(n_samples, dtype=bool)
    indices = random_state.randint(0, n_samples, n_samples_bootstrap)
    sampled_mask[indices] = True
    unsampled_mask = ~sampled_mask
    indices_range = np.arange(n_samples)
    unsampled_indices = indices_range[unsampled_mask]
    return unsampled_indices

# Mock version of _estimate_recurrent_concordance_index
def _estimate_recurrent_concordance_index(time_stop, event, predictions):
    # This is a mock version, so we are returning a dummy value for now.
    return 0.5

# Attach these methods to the RecurrentRandomForest class
setattr(RecurrentRandomForest, "_validate_params", _validate_params)
setattr(RecurrentRandomForest, "_validate_data", _validate_data)
setattr(RecurrentRandomForest, "_validate_estimator", _validate_estimator)
setattr(RecurrentRandomForest, "_make_estimator", _make_estimator)
setattr(RecurrentRandomForest, "_validate_X_predict", _validate_X_predict)

# Generate synthetic recurrent survival data for testing
np.random.seed(0)
n_samples = 100
n_features = 5

# Generate random data
X = np.random.randn(n_samples, n_features)
ids = np.arange(n_samples)
time_start = np.random.rand(n_samples) * 5
time_stop = time_start + np.random.rand(n_samples) * 5
event = np.random.randint(0, 2, n_samples)

X, ids, time_start, time_stop, event


In [ ]:
class RecurrentTree:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1, random_state=None):
        pass

    def fit(self, X, ids, time_start, time_stop, event, sample_weight=None, max_samples=None):
        return self

    def predict_rate_function(self, X):
        return np.random.rand(X.shape[0], 10)

    def predict_mean_function(self, X):
        return np.random.rand(X.shape[0], 10)

rrf = RecurrentRandomForest(max_depth=10, random_state=1190)
# Fit the model to the synthetic data
rrf.fit(X, ids, time_start, time_stop, event)

# Predict using the model
rate_predictions = rrf.predict_rate_function(X[:5])
mean_predictions = rrf.predict_mean_function(X[:5])

rate_predictions, mean_predictions